In [1]:
from database_app import query_table

#### Query sample records

In [2]:
sample_df = query_table(
    """
    select date_time, lower(word), word_count from twitter_words limit 3
    """
)

2.6.0


In [3]:
sample_df.head(3)

,date_time,lower(word),word_count
0,2020-07-05 19:29:43,@fanetope,1
1,2020-07-05 19:29:43,be,1
2,2020-07-05 19:29:43,me,1


#### Duration of stored twitter data

In [4]:
query_table(
    """
    select
        substr(date_time, 1, 10) as stream_date,
        min(substr(date_time, 11, 18)) as start_time,
        max(substr(date_time, 11, 18)) as end_time,
        count(*) as total_tweets
    from twitter_words
    group by stream_date
    """
)

2.6.0


,stream_date,start_time,end_time,total_tweets
0,2020-07-05,19:29:43,21:17:57,10194


#### Query top 5 words (ignore stop words)

In [5]:
stopwords = [
    'the', 'to', 'i', 'a', 'on', 'is', 'in', 'you', 'of', 'it', 'and', 'this', 'was', 'he', 'had', 'our', 'from', 'what',
    'for', 'that', 'be', 'my', 'so', 'they', 'all', 'have', 'me', 'his', 'with', 'when', 'we', 'but', 'not', 'are', 'at',
    'no', 'now', 'your', 'there', 'just', 'can', 'will', 'like', 'if', 'am', 'it’s', 'i’m'
]

stopwords.sort()

In [6]:
# SQLite does not provide regexp related functions

query_table(
    """
    select
        lower(word),
        sum(word_count)
    from twitter_words
    where lower(word) not in ({}) and trim(word) != '' and trim(word) not in ('@', '.', '(', ',')
    group by lower(word)
    order by sum(word_count) desc
    limit 5
    """.format(str(stopwords)[1:-1])
)

2.6.0


,lower(word),sum(word_count)
0,happy,138
1,july,79
2,new,70
3,4,69
4,city,68


#### Query top 5 tagged individuals

In [7]:
query_table(
    """
    select word as tagged, sum(word_count) as tagged_freq
    from twitter_words
    where word like '@%' and word != '@'
    group by word
    order by tagged_freq desc
    limit 5
    """
)

2.6.0


,tagged,tagged_freq
0,@realDonaldTrump,20
1,@kanyewest,7
2,@greggutfeld,5
3,@WhiteHouse,5
4,@FordFischer,5


#### Query top 5 hashtag

In [8]:
query_table(
    """
    select word as hashtag, sum(word_count) as hashtag_freq
    from twitter_words
    where word like '#%'
    group by word
    order by hashtag_freq desc
    limit 5
    """
)

2.6.0


,hashtag,hashtag_freq
0,#7Line,12
1,#fireworks,6
2,#DJ_JOHNNY_SUNNYSIDEUP,5
3,#Corinthians:,5
4,#Bono:,5
